In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from os import path, environ
from datetime import datetime
from services.zayev.environment.market_simulator import MarketSimulator
from services.zayev.service_layer.forester import Forester
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import accuracy_score


2023-12-10 05:32:48.297093: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:


starting_time_step = datetime(year=1995, month=1, day=1, hour=10)

db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
env_config = {
    "db_params": db_params, 
    "max_episode_steps": 1000, 
    "the_current_time_step": starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": True
} 

market = MarketSimulator(env_config=env_config)

In [3]:
market.reset()

(array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]))

In [4]:
forest = Forester(market=market)
print("done")

done


In [5]:
forest.make_forest_data()

In [6]:
X_train = forest.train_input
y_train = forest.train_output
y_test = forest.test_output
X_test = forest.test_input

print(X_train[9])
# print(X_test)
print(y_train[8])
# print(y_test)

[-1.29080310e-02 -7.57105219e-03 -1.62929304e-03 -1.16672352e-03
  1.63131116e-02  1.84732531e-03 -5.57881967e-03  9.55223357e-03
  7.29820962e-03  1.62479197e-02  6.74165694e-03 -7.04839883e-03
 -4.55755739e-02 -4.45162225e-02 -5.46539683e-02  3.48319067e-03
  6.13891037e-03  6.19844181e-03  1.46138027e-02 -1.01126905e-02
  1.04352002e-02  1.78105042e-02 -7.35834346e-03 -1.02725985e-03
  8.75760649e-03 -2.03242558e-02 -3.55360208e-03  3.19915527e-02
  3.91445421e-02  2.85282289e-02 -3.40803669e-03 -4.40754751e-04
  1.83065726e-02  1.73662478e-02  1.92435078e-03 -4.34676681e-03
 -8.46276602e-03  3.37578649e-02  3.57776237e-02  1.97667973e-02
 -9.86161586e-03 -1.92699408e-03  2.46194882e-02  2.06063293e-02
  2.32773301e-02  9.09141519e-03 -1.18774900e-02 -2.09352902e-02
 -1.77393948e-02  8.50242124e-03 -2.45983759e-03  1.11713703e-02
  7.92924828e-03  1.91343096e-02 -1.04378462e-02  9.40707227e-04
  9.64974681e-04  1.49986359e-03  1.56791994e-03  1.86584757e-03
  1.36374261e-02  8.76363

In [7]:
# rf = RandomForestClassifier()
# rf.fit(X_train, y_train)

In [10]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1200, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 80, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

for n_ests in n_estimators:
    for max_feat in max_features:
        for max_dept in max_depth:
            for min_sample_sp in min_samples_split:
                for min_sample_lf in min_samples_leaf:
                    for bt_strp in bootstrap:
                        rf = RandomForestClassifier(
                            n_estimators=n_ests,
                            max_features=max_feat,
                            max_depth=max_dept,
                            min_samples_split=min_sample_sp,
                            min_samples_leaf=min_sample_lf,
                            bootstrap=bt_strp
                        )
                        rf.fit(X_train, y_train)
                        X_test = forest.test_input
                        y_pred = rf.predict(X_test)
                        y_test = forest.test_output
                        y_test_first_column = [row[0] for row in y_test]
                        y_pred_first_column = [row[0] for row in y_pred]
                        accuracy = accuracy_score(y_test_first_column, y_pred_first_column)
                        print("Accuracy:", accuracy)

[200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]


In [8]:
X_test = forest.test_input
# X_test = forest.train_input
y_pred = rf.predict(X_test)

In [16]:
print(y_pred[:, 0])

[10. 12. 10. 11. 10. 10. 11. 13. 11. 10. 10. 11. 11. 12. 12. 12. 11. 10.
 11. 10.  9.  9.  8. 10. 11. 11. 11.  9.  9.  9.  9. 11. 10. 11. 11. 11.
 10. 10.  7.  7.  9. 11. 10. 11. 10. 10. 11. 11. 11. 10. 11.  7.  6. 10.
  9.  9.  9. 12. 11. 13. 10. 10. 11. 11. 10. 12. 10.  9. 11.  9. 12. 10.
 12. 10. 10.  7.  7.  9.  6. 21. 11. 10. 10. 11. 10.  9. 11. 10. 11.  7.
  9.  9. 21. 11. 12.  7. 11. 10. 11. 11. 11. 11.  9.  9.  8.  9.  8.  9.
 11. 11. 11. 10.  9. 11. 11. 10. 10. 11. 10. 10. 11. 11. 11. 11. 11. 10.
 10. 11. 10. 11. 11.  9. 10. 11. 11. 11. 11. 10. 10. 11. 10. 10. 11.  9.
 10. 10. 10.  9. 10. 10. 11. 11. 11. 10. 10. 11. 10. 11. 11.  9.  9. 11.
 10. 10. 10.  9.  9. 10. 10. 10.  9.  9. 10. 11.  8. 10. 11. 10. 10. 10.
 10. 10. 10. 10. 10.  9. 10. 10. 10. 10. 10. 12. 11. 12. 11. 10. 11. 10.
  9. 10.  7.  8.  7.  8. 11.  8.  8.  8.  8. 10.  8. 11. 10.]


In [9]:
y_test = forest.test_output
# y_test = forest.train_output
y_test_first_column = [row[0] for row in y_test]
y_pred_first_column = [row[0] for row in y_pred]

In [10]:
y_test = forest.test_output
accuracy = accuracy_score(y_test_first_column, y_pred_first_column)
print("Accuracy:", accuracy)

Accuracy: 0.18309859154929578


In [44]:

from matplotlib.dates import relativedelta


nw_starting_time_step = datetime(year=1995, month=1, day=1, hour=10) + relativedelta(hours=1000)

nw_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nw_env_config = {
    "db_params": nw_db_params, 
    "max_episode_steps": 72, 
    "the_current_time_step": nw_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": True
} 

nw_market = MarketSimulator(env_config=nw_env_config)
nw_market.reset()
nw_forest = Forester(market=nw_market)

In [45]:
nw_forest.make_forest_data()

In [46]:
nw_X_train = nw_forest.train_input
nw_y_train = nw_forest.train_output
nw_y_test = nw_forest.test_output
nw_X_test = nw_forest.test_input

nw_X_test = nw_forest.test_input
nw_y_pred = rf.predict(nw_X_test)

nw_y_test = nw_forest.test_output
nw_y_test_first_column = [row[1] for row in nw_y_test]
nw_y_pred_first_column = [row[1] for row in nw_y_pred]

In [47]:
accuracy = accuracy_score(nw_y_test_first_column, nw_y_pred_first_column)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [49]:
print(nw_forest.market.the_current_time_step)
print(nw_y_pred)

1995-02-19 14:00:00+00:00
[[ 8. 10. 12. 13. 11.]
 [11.  9. 10. 10. 11.]
 [ 0. 16. 21. 19.  3.]
 [ 1. 16. 21. 21.  5.]
 [ 9. 12. 12. 13. 10.]
 [21.  8.  6.  9. 14.]
 [13.  8.  4.  6. 12.]
 [10. 10.  9.  9. 10.]
 [ 8. 12. 17. 13.  9.]
 [11.  9.  7.  7. 11.]]


In [15]:
time_data = {}
for it in range(5):
    rand_acts = nw_forest.act_random()
    state, reward, done, info = nw_forest.market.step([rand_acts])
    time_data[nw_forest.market.the_current_time_step] = state
nw_state = state
print(nw_forest.market.the_current_time_step)
flat_state = np.array([nw_forest.get_flattened_states(nw_state)])
my_pred = rf.predict(flat_state)
print(len(nw_y_pred), my_pred)

1995-03-05 10:30:00+00:00
38 [[13.  9. 10.  7. 12.]]
